# Asynch adata converter to msg 

In [3]:
%matplotlib inline 
from ifis_tools import asynch_manager as am 
from ifis_tools import database_tools as db
import pandas as pd
import numpy as np
import glob
from multiprocessing import Pool

In [2]:
USGS_links = db.SQL_USGS_at_IFIS()

In [3]:
L = glob.glob('/media/nicolas/Data/RunoffExp/Raw/190_*.dat')
L.sort()

In [9]:
LinksID = []
for k in USGS_links.keys():
    LinksID.append(USGS_links[k])

In [ ]:
Data = {}
for l in L:
    #try:
    d = am.ASYNCH_results(l)
    for link in LinksID[87:]:
        try:
            Data.update({str(link)+'_'+l.split('_')[1][:4] : d.ASYNCH_dat2Serie(link, l.split('_')[1][:4]+'-04-01 00:00', freq = '15min')})
        except:
            pass
    print(l.split('_')[1][:4])

In [ ]:
dates = pd.date_range('2008-04-01','2018-12-01', freq = '15min')
for link in LinksID[87:]:
    try:
        Lista = []
        for ano in L:
            Lista.append(Data[str(link)+'_'+l.split('_')[1][:4]])
        D = pd.concat(Lista)
        D2 = pd.Series(np.zeros(dates.size), dates)
        D2[D.index] = D.values
        D2[D2 == 0] = np.percentile(D.values,10)
        D2.to_msgpack('/media/nicolas/Data/RunoffExp/Results/HLM190/'+str(link)+'_190.msg')
        print(link)    
    except:
        pass

# Mean rainfall for each link

In [7]:
def warp(link):
    R =  db.SQL_Get_MeanRainfall(link, '2008-04-01','2018-12-1')
    R.to_msgpack('/media/nicolas/Data/RunoffExp/Results/Rainfall/Rain_'+str(link)+'.msg')

In [11]:
%%time
p = Pool(processes=7)
p.map(warp, LinksID[14:])
p.close()
p.join()


CPU times: user 961 ms, sys: 482 ms, total: 1.44 s
Wall time: 18min 15s
